In [1]:
pip install llama-index langchain pypdf langchain-community llama-index-llms-huggingface-api llama-index-embeddings-langchain


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from getpass import getpass
HF_TOKEN = getpass()

 ········


In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext,load_index_from_storage
from llama_index.core.settings import Settings
from llama_index.core.node_parser import SimpleNodeParser
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [27]:
lectures = []
for filename in os.listdir('sources'):
    filepath = os.path.join('sources', filename)
    with open(filepath, "r", encoding="utf-8") as f:
        lectures.append(f.read())

In [28]:
#chunk text
chunks = []
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

for lecture in lectures:
    chunks.extend(splitter.split_text(lecture))

In [29]:
chunks[:10]

['\\begin{definition}\n    Статистика $S(X)$ называется \\textit{полной} для $\\theta$, если из условия $E_\\theta f(S(X)) = 0\\ \\forall \\theta \\in \\Theta$ следует, что $f(S(X)) = 0\\ \\forall \\theta \\in \\Theta\\ P_\\theta-a.s.$\n\\end{definition}\n\\begin{theorem}',
 '\\end{definition}\n\\begin{theorem}\n    (Лемана-Шеффе, об оптимальной оценке) Пусть $\\displaystyle T( X)$ -- полная достаточная статистика для $\\displaystyle \\{P_{\\theta } ,\\ \\theta \\in \\Theta \\} ,\\ \\phi ( T( X))$ -- несмещенная оценка для $\\displaystyle \\tau ( \\theta )$. Тогда $\\displaystyle \\phi ( T( X))$ не хуже любой несмещенной оценки $\\displaystyle \\tau ( \\theta )$ в среднеквадратичном подходе, и если $\\displaystyle \\phi ( T) \\in L_{2}$, то $\\displaystyle \\phi ( T)$ -- оптимальная оценка.',
 '\\end{theorem}\n\\begin{proof}',
 'Пусть $\\displaystyle \\tilde{d}$ -- несмещенная оценка $\\displaystyle \\tau ( \\theta )$. Тогда $\\displaystyle \\tilde{\\phi }( T) =E\\left(\\tilde{d} \\ |\

In [30]:
from llama_index.core.schema import TextNode

nodes = []

for chunk in chunks:
    node = TextNode(text=chunk)
    nodes.append(node)

In [31]:
rm -r ./storage

rm: ./storage: No such file or directory


In [32]:
#create index
Settings.llm = HuggingFaceInferenceAPI(
    model_name="meta-llama/llama-3.2-1B", token=HF_TOKEN
)
Settings.embed_model = LangchainEmbedding(
    HuggingFaceInferenceAPIEmbeddings(api_key=HF_TOKEN,model_name="sentence-transformers/all-MiniLM-L6-v2")
)
storage_context = StorageContext.from_defaults()
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
)
index.storage_context.persist(persist_dir="./storage")

In [54]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

response_synthesizer = get_response_synthesizer(response_mode="compact")
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

In [58]:
nodes = vector_retriever.retrieve("квантиль")

In [59]:
nodes[0].text

'(Будет позже)\n\\subsection{Метод выборочных квантилей}'

In [60]:
nodes[1].text

'статистика. Пусть $\\displaystyle u_{p}$ -- $\\displaystyle p$-квантиль стандартного нормального распределения. Тогда'

In [38]:
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

In [43]:
print(vector_query_engine.query("Что такое квантиль?"))

 Квантиль -- это значение, которое делит распределение вероятностей на части с равными вероятностями. Например, $\displaystyle u_{p}$ -- это $\displaystyle p$-квантиль стандартного нормального распределения, что означает, что $\displaystyle P(X \leq u_{p}) = p$, где $\displaystyle X$ -- случайная величина, имеющая стандартное нормальное распределение. Таким образом, квантиль $\displaystyle u_{p}$ -- это значение, ниже которого лежит $\displaystyle p$-я доля распределения. Например, медиана (0.5-квантиль) стандартного нормального распределения равна 0, так как 50% значений лежат ниже 0. 

Если у вас есть дополнительные вопросы или нужна более подробная информация, дайте знать! 😊
--------------------- To provide a more precise and concise answer based on the given context:

**Квантиль** -- это значение, которое делит распределение вероятностей на части с равными вероятностями. Например, \( u_p
